# 4 - Vlasov-Ampère equations

The equations we will solve are described in the model [VlasovAmpereOneSpecies](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_kinetic.html#struphy.models.kinetic.VlasovAmpereOneSpecies).
To create the default parameter file from the console:

```
struphy params VlasovAmpereOneSpecies
```

Adapt the parameters and run the model with

```
python3 params_VlasovAmpereOneSpecies.py
```

In this notebook we shall re-create the parameter file and perform some tests.

## Weak Landau damping

1. Imports:

In [ ]:
from struphy import main
from struphy.fields_background import equils
from struphy.geometry import domains
from struphy.initial import perturbations
from struphy.io.options import BaseUnits, DerhamOptions, EnvironmentOptions, FieldsBackground, Time
from struphy.kinetic_background import maxwellians
from struphy.models.kinetic import VlasovAmpereOneSpecies
from struphy.pic.utilities import BinningPlot, BoundaryParameters, LoadingParameters, WeightsParameters
from struphy.topology import grids

2. Generic options:

In [ ]:
# environment options
env = EnvironmentOptions()

# units
base_units = BaseUnits()

# time stepping
time_opts = Time(dt=0.05, Tend=0.5)  # , Tend = 3.5

# geometry
r1 = 12.56
domain = domains.Cuboid(r1=r1)

# fluid equilibrium (can be used as part of initial conditions)
equil = None

# grid
grid = grids.TensorProductGrid(Nel=(32, 1, 1))

# derham options
derham_opts = DerhamOptions()

3. Model instance and physics parameters:

In [ ]:
model = VlasovAmpereOneSpecies()

model.kinetic_ions.set_phys_params(alpha=1.0, epsilon=1.0)

4. Kinetic species parameters:

In [ ]:
loading_params = LoadingParameters(ppc=10000)
weights_params = WeightsParameters(control_variate=True)
boundary_params = BoundaryParameters()
model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params
)
model.kinetic_ions.set_sorting_boxes()

In [ ]:
# particle binning
binplot_1 = BinningPlot(slice="e1", n_bins=128, ranges=(0.0, 1.0))
binplot_2 = BinningPlot(slice="v1", n_bins=128, ranges=(-5.0, 5.0))
binplot_3 = BinningPlot(slice="e1_v1", n_bins=(128, 128), ranges=((0.0, 1.0), (-5.0, 5.0)))

binning_plots = (binplot_1, binplot_2, binplot_3)

model.kinetic_ions.set_save_data(binning_plots=binning_plots)

5. Propagator options:

In [ ]:
model.propagators.push_eta.options = model.propagators.push_eta.Options()
model.propagators.coupling_va.options = model.propagators.coupling_va.Options()
model.initial_poisson.options = model.initial_poisson.Options(stab_eps=1e-12)

6. Initial conditions:

In [ ]:
background = maxwellians.Maxwellian3D(n=(1.0, None))
model.kinetic_ions.var.add_background(background)

perturbation = perturbations.ModesCos(ls=[1], amps=[0.001])
init = maxwellians.Maxwellian3D(n=(1.0, perturbation))
model.kinetic_ions.var.add_initial_condition(init)

Let us run the model. However, depending on the confguration, running in a notebook might be very slow. In order to get fast execution, run from the console. First, create the default parameter file and rename it 

```
struphy params VlasovAmpereOneSpecies
mv params_VlasovAmpereOneSpecies.py landau.py
```

Adapt it with the parameters from this notebook. Start the run with

```
python landau.py
```

or 

```
mpirun -n 2 python landau.py
```

for a run on two threads. Line profiling can be enabled with 

```
LINE_PROFILE=1 mpirun -n 2 landau.py
```

Let us look a the slower run in the notebook:

In [ ]:
verbose = True

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

In [ ]:
import os

path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path, celldivide=8)

In [ ]:
simdata = main.load_data(path)

In [ ]:
# plot in v1
from matplotlib import pyplot as plt

v1_bins = simdata.f["kinetic_ions"]["v1"]["grid_v1"]
f_v1_init = simdata.f["kinetic_ions"]["v1"]["f_binned"][0]

plt.plot(v1_bins, f_v1_init)
plt.xlabel("vx")
plt.title("Initial Maxwellian");

In [ ]:
# plot in e1

e1_bins = simdata.f["kinetic_ions"]["e1"]["grid_e1"]
df_e1_init = simdata.f["kinetic_ions"]["e1"]["delta_f_binned"][0]

plt.plot(e1_bins, df_e1_init)
plt.xlabel("$\eta_1$")
plt.title("Initial spatial perturbation");

In [ ]:
# plot in e1-v1

e1_bins = simdata.f["kinetic_ions"]["e1_v1"]["grid_e1"]
v1_bins = simdata.f["kinetic_ions"]["e1_v1"]["grid_v1"]
f_init = simdata.f["kinetic_ions"]["e1_v1"]["f_binned"][0]
df_init = simdata.f["kinetic_ions"]["e1_v1"]["delta_f_binned"][0]
f_end = simdata.f["kinetic_ions"]["e1_v1"]["f_binned"][-1]
df_end = simdata.f["kinetic_ions"]["e1_v1"]["delta_f_binned"][-1]

plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.pcolor(e1_bins, v1_bins, f_init.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Initial Maxwellian")
plt.colorbar()

plt.subplot(2, 2, 2)
plt.pcolor(e1_bins, v1_bins, df_init.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Initial perturbation")
plt.colorbar()

plt.subplot(2, 2, 3)
plt.pcolor(e1_bins, v1_bins, f_end.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Final Maxwellian")
plt.colorbar()

plt.subplot(2, 2, 4)
plt.pcolor(e1_bins, v1_bins, df_end.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Final perturbation")
plt.colorbar();

In [ ]:
# electric field

e1, e2, e3 = simdata.grids_log
e_vals = simdata.spline_values["em_fields"]["e_field_log"][0][0]

plt.plot(e1, e_vals[:, 0, 0], label="E")
plt.xlabel("$\eta_1$")
plt.title("Initial electric field")
plt.legend();